# Import
该笔记记录了各种特征的提取方案，被一种特征提取会在最后总结为一个提取函数，其输入是DRT结果，输出的评估特征组成的向量

In [1]:
import os
import re
import gc
import sys 
from datetime import datetime
from loguru import logger



import torch
import numpy as np


import matplotlib.pyplot as plt
%matplotlib qt




# Feature Extraction

## Definition

In [10]:
def SearchELE(rootPath, ele_pattern = re.compile(r"(.+?)_归档")):
    '''==================================================
        Search all electrode directories in the rootPath
        Parameter: 
            rootPath: current search path
            ele_pattern: electrode dir name patten
        Returen:
            ele_list: list of electrode directories
        ==================================================
    '''
    ele_list = []
    for i in os.listdir(rootPath):
        _path = os.path.join(rootPath, i)
        if os.path.isdir(_path):
            match_ele = ele_pattern.match(i)
            if match_ele:
                ele_list.append([_path, match_ele.group(1)])
            else:
                ele_list.extend(SearchELE(_path, ele_pattern))

    return ele_list



def Load_Single(ele_id, rootPath, DATA_SUFFIX):
    fd_pt = os.path.join(f"{rootPath}/{ele_id}_归档", DATA_SUFFIX, f"{ele_id}_{DATA_SUFFIX}.pt")
    if not os.path.exists(fd_pt):
        logger.warning(f"{fd_pt} does not exist")
        return None
    data_pt = torch.load(fd_pt, weights_only=False)
    _data_group = data_pt["data_group"]

    return _data_group



def DRT_Plot_Batch(fig, DRTdata_list, EISdata_list, Loess_list, eis_seq):
    
    axis = [0] * 6
    axis[0] = fig.add_subplot(2,3,1)    # Nyquist Plot
    axis[1] = fig.add_subplot(2,3,2)    # Bode Plot (Magnitude)
    axis[2] = fig.add_subplot(2,3,3)    # Bode Plot (Phase)
    axis[3] = fig.add_subplot(2,3,4)    # Text
    axis[4] = fig.add_subplot(2,3,5)    # DRT (RC)
    axis[5] = fig.add_subplot(2,3,6)    # DRT (Rτ)


    text_axis = axis[3]
    text_axis.axis('off')

    _s       = 2
    _alpha   = 0.7

    cmap = plt.colormaps.get_cmap('rainbow_r')
    for i in range(len(EISdata_list)):
        if i in eis_seq:
            ch_eis      = EISdata_list[i][0]
            ch_drt      = DRTdata_list[i]
            ch_loess    = Loess_list[i]

            # ch_R    = np.array([i[1:,0] for i in ch_drt])
            # ch_C    = np.array([i[1:,-1] for i in ch_drt])
            ch_R    = np.concatenate([i[1,:] for i in ch_drt])
            ch_C    = np.concatenate([i[2,:] for i in ch_drt])

            _color  = cmap(i/len(EISdata_list))

            axis[0].plot(ch_eis[1,:], -ch_eis[2,:], color = _color, linewidth=2)
            axis[1].loglog(ch_eis[0,:], np.abs(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2)
            axis[2].semilogx(ch_eis[0,:], np.rad2deg(np.angle(ch_eis[1,:]+1j*ch_eis[2,:])), color = _color, linewidth=2)

            axis[4].scatter(ch_R, ch_C, s=_s, alpha=_alpha, color=_color, label=f'ch[{i:03d}]')
            axis[5].scatter(ch_loess[0,:], ch_loess[1,:], s=_s, alpha=_alpha, color=_color, label=f'ch[{i:03d}]')



    axis[0].set_aspect('equal', adjustable='datalim')
    axis[4].set_xscale('log')
    axis[4].set_yscale('log')
    axis[5].set_xscale('log')
    axis[5].set_yscale('log')


    return text_axis




## Load Data

In [219]:

rootPath = "D:/Baihm/EISNN/Archive/"
ele_list = SearchELE(rootPath)
DATASET_SUFFIX = "Outlier_Ver04"

# rootPath = "D:/Baihm/EISNN/Archive_New/"
# ele_list = SearchELE(rootPath)
# DATASET_SUFFIX = "Outlier_Ver04"

# rootPath = "D:/Baihm/EISNN/Invivo/"
# ele_list = SearchELE(rootPath, re.compile(r"(.+?)_Ver02"))
# DATASET_SUFFIX = "Outlier_Ver04"


n_ele = len(ele_list)
logger.info(f"Search in {rootPath} and find {n_ele:03d} electrodes")

2025-07-18 16:31:05.194 | INFO     | __main__:<module>:15 - Search in D:/Baihm/EISNN/Archive/ and find 218 electrodes


In [292]:
ele_id = '06017758'
ch_id = 96     # Perfect of Perfect

# ele_id = '09290511'
# ch_id = 13    # Up & Down, 2 outliers
# ch_id = 21    # Normal + 2 outlier
# ch_id = 41    # Normal + 2 outlier - *(Hard To Tell)
# ch_id = 79    # 3-class, What a mess


# ele_id = '01037160'
# ch_id = 20  # Normal to Short, Same to GPR  
# ch_id = 89  # Same to GPR  
# ch_id = 7  # Normal Example


# ele_id = '05087163'
# ch_id = 7   # one outlier
# ch_id = 50  # No outlier but in two Phases
# ch_id = 55  # One outlier &wired end point
# ch_id = 114 # Open Circuit with on outpler



# ele_id = '02067447'
# ch_id = 68  # Short all the time


# ele_id = '11057712'
# ch_id = 106    # Very Good Electrode with 1 hidden outlier, and one phase shift



# ele_id = '10057084'
# ch_id = 16    # Totaly Mess
# ch_id = 18    # Totaly Mess




Loe_Data = Load_Single(ele_id, rootPath, DATA_SUFFIX = f"{DATASET_SUFFIX}_DRTLoe_Ver02")
Loe_Data = Loe_Data[ch_id]



DRTdata_list    = Loe_Data['DRTlist']  
EISdata_list    = Loe_Data['EISlist']  
Loess_list      = Loe_Data['Loesslist']
eis_seq         = Loe_Data['eis_seq']     
eis_short       = Loe_Data['seq_short']  


fig = plt.figure(figsize=(16, 9), constrained_layout=True)
text_axis = DRT_Plot_Batch(fig, DRTdata_list, EISdata_list, Loess_list, eis_seq)



## RC
目前倾向于linear space下使用trim mean

### Draft

In [222]:
Draft_Flag = False
if Draft_Flag is True:
    def feature_Rinf(DRTdata_list):
        '''==================================================
            Calculate Rinf from DRTdata_list
            Parameter: 
                DRTdata_list: list of DRT data - day x sample x [T,R,C]
            Return:
                Rinf_list: list of Rinf values
            ==================================================
        '''
        Rinf_list = []
        C0_list = []
        for ch_drt in DRTdata_list:
            Rinf = [i[:2,0] for i in ch_drt]
            C0 = [i[[0,2],-1] for i in ch_drt]
            Rinf_list.append(Rinf)
            C0_list.append(C0)

        # return np.concatenate(Rinf_list)
        return np.array(Rinf_list), np.array(C0_list)

    def exp_weighted_mean(t, X, lam=0.1):
        weights = np.exp(-lam * t)
        weights /= np.sum(weights)
        return np.sum(weights * X)



    def tukey_weights(data, c=6.0):
        median = np.median(data)
        mad = np.median(np.abs(data - median))
        u = (data - median) / (c * mad + 1e-8)
        weights = (1 - u**2)**2
        weights[np.abs(u) >= 1] = 0
        return data[weights]

    def tukey_biweight_location(data, c=6.0, epsilon=1e-10):
        data = np.asarray(data)
        M = np.median(data)  # 初始位置：中位数
        MAD = np.median(np.abs(data - M)) + epsilon  # 避免除0

        u = (data - M) / (c * MAD)
        u[np.abs(u) >= 1] = 0  # 超出范围的 u 设置为0
        w = (1 - u**2)**2

        # 加权平均
        numerator = np.sum(w * data)
        denominator = np.sum(w) + epsilon
        return numerator / denominator

    def bootstrap_trimmed_mean(data, proportion_to_cut=0.1, n_boot=1000):
        n = len(data)
        estimates = []
        for _ in range(n_boot):
            resample = np.random.choice(data, size=n, replace=True)
            resample_sorted = np.sort(resample)
            k = int(n * proportion_to_cut)
            trimmed = resample_sorted[k:n-k]
            estimates.append(np.mean(trimmed))
        return np.mean(estimates), np.std(estimates, ddof=1)  # 均值，标准误

    Rinf_list, C0_list = feature_Rinf([DRTdata_list[i] for i in eis_seq])
    # Rinf_list = feature_Rinf(DRTdata_list)
    Rinf_list.shape
    C0_list.shape


    fig = plt.figure()
    axis = fig.add_subplot(111)
    cmap = plt.colormaps.get_cmap('rainbow_r')
    for i in range(Rinf_list.shape[0]):
        if i != 2: continue
        _color = cmap(i/Rinf_list.shape[0])
        # axis.scatter(Rinf_list[i,:,0], Rinf_list[i,:,1], s=10, alpha=1, color=_color)
        axis.scatter(Rinf_list[i,:,0], Rinf_list[i,:,1], s=10, alpha=1, color='gray')
        # axis.scatter(C0_list[i,:,0], C0_list[i,:,1], s=10, alpha=1, color=_color)


        mu = exp_weighted_mean(np.log(Rinf_list[i,:,0]), np.log(Rinf_list[i,:,1]))
        # mu = exp_weighted_mean(np.log(1/C0_list[i,:,0]), np.log(C0_list[i,:,1]))

        bw_mean_log = tukey_biweight_location(np.log(Rinf_list[i,:,1]), c=10)
        bw_mean_org = tukey_biweight_location(Rinf_list[i,:,1], c=10)
        
        from scipy.stats import trim_mean
        trim_mu_log = trim_mean(np.log(Rinf_list[i,:,1]), proportiontocut=0.1)
        trim_mu_org = trim_mean(Rinf_list[i,:,1], proportiontocut=0.3)

        # trim_mu, trim_std = bootstrap_trimmed_mean(Rinf_list[i,:,1])
        trim_mu, trim_std = bootstrap_trimmed_mean(np.log(Rinf_list[i,:,1]))

        # axis.axhline(np.exp(mu), color=_color, linestyle='--', alpha=0.5, label=f'ch[{eis_seq[i]:03d}]')

        # axis.axhline(np.exp(np.mean(np.log(Rinf_list[i,:,1]))), color='red', linestyle='--', alpha=0.8, label=f'Mean Log')
        # axis.axhline(np.exp(np.median(np.log(Rinf_list[i,:,1]))), color='orange', linestyle='--', alpha=0.8, label=f'Median Log')
        # axis.axhline(np.exp(bw_mean_log), color='blue', linestyle='--', alpha=0.8, label=f'Biweight Mean Log')
        # axis.axhline(np.exp(trim_mu_log), color='green', linestyle='--', alpha=0.8, label=f'Trim Mean Log')


        # axis.axhline(np.mean(Rinf_list[i,:,1]), color='red', linestyle='--', alpha=0.8, label=f'Mean')
        # axis.axhline(np.median(Rinf_list[i,:,1]), color='orange', linestyle='--', alpha=0.8, label=f'Median')
        # axis.axhline(bw_mean_org, color='blue', linestyle='--', alpha=0.8, label=f'Biweight Mean')
        # axis.axhline(trim_mu_org, color='green', linestyle='--', alpha=0.8, label=f'Trim Mean')

        
        # axis.axhline(trim_mu, color='green', linestyle='--', alpha=0.8, label=f'Trim Mean')
        # axis.axhline(trim_mu+3*trim_std, color='red', linestyle='--', alpha=0.8, label=f'Trim std x 3')
        # axis.axhline(trim_mu-3*trim_std, color='red', linestyle='--', alpha=0.8)

        
        axis.axhline(np.exp(trim_mu), color='green', linestyle='--', alpha=0.8, label=f'Trim Mean')
        axis.axhline(np.exp(trim_mu+3*trim_std), color='red', linestyle='--', alpha=0.8, label=f'Trim std x 3')
        axis.axhline(np.exp(trim_mu-3*trim_std), color='red', linestyle='--', alpha=0.8)

        # axis.plot(Rinf_list[i,:,0], Rinf_list[i,:,1], color=_color)
    axis.set_xlabel('Rinf')
    axis.set_ylabel('Rinf Value')
    axis.set_title('Rinf Values from DRT Data')
    axis.set_xscale('log')
    axis.set_yscale('log')
    axis.legend()



### Result

In [223]:

from scipy.stats import trim_mean
def bootstrap_trimmed_mean(data, proportion_to_cut=0.1, n_boot=1000):
    n = len(data)
    estimates = []
    for _ in range(n_boot):
        resample = np.random.choice(data, size=n, replace=True)
        resample_sorted = np.sort(resample)
        k = int(n * proportion_to_cut)
        trimmed = resample_sorted[k:n-k]
        estimates.append(np.mean(trimmed))
    return np.mean(estimates), np.std(estimates, ddof=1) 

In [266]:

Rinf_list, C0_list = feature_Rinf([DRTdata_list[i] for i in eis_seq])


fig = plt.figure()
axis1 = fig.add_subplot(121)
axis2 = fig.add_subplot(122)
cmap = plt.colormaps.get_cmap('rainbow_r')
for i in range(Rinf_list.shape[0]):
    if i != 7: continue
    _color = cmap(i/Rinf_list.shape[0])
    axis1.scatter(Rinf_list[i,:,0], Rinf_list[i,:,1], s=10, alpha=1, color=_color)
    axis2.scatter(C0_list[i,:,0], C0_list[i,:,1], s=10, alpha=1, color=_color)

    # axis.scatter(Rinf_list[i,:,0], Rinf_list[i,:,1], s=10, alpha=1, color='gray')

    
    # trim_mu_Rinf    = trim_mean(Rinf_list[i,:,1], proportiontocut=0.4)
    # trim_mu_C0      = trim_mean(C0_list[i,:,1], proportiontocut=0.4)
    # axis1.axhline(trim_mu_Rinf, color=_color, linestyle='--', alpha=0.8, label=f'Trim Mean')
    # axis2.axhline(trim_mu_C0, color=_color, linestyle='--', alpha=0.8, label=f'Trim Mean')

    trim_R_mu, trim_R_std = bootstrap_trimmed_mean(Rinf_list[i,:,1], proportion_to_cut=0.3)
    trim_C_mu, trim_C_std = bootstrap_trimmed_mean(C0_list[i,:,1], proportion_to_cut=0.3)

    axis1.axhline(trim_R_mu, color=_color, linestyle='--', alpha=0.8, label=f'Trim Mean')
    axis1.axhline(trim_R_mu+3*trim_R_std, color=_color, linestyle='--', alpha=0.8, label=f'Trim std x 3')
    axis1.axhline(trim_R_mu-3*trim_R_std, color=_color, linestyle='--', alpha=0.8)

    axis2.axhline(trim_C_mu, color=_color, linestyle='--', alpha=0.8, label=f'Trim Mean')
    axis2.axhline(trim_C_mu+3*trim_C_std, color=_color, linestyle='--', alpha=0.8, label=f'Trim std x 3')
    axis2.axhline(trim_C_mu-3*trim_C_std, color=_color, linestyle='--', alpha=0.8)

    

    # axis.plot(Rinf_list[i,:,0], Rinf_list[i,:,1], color=_color)
axis1.set_xlabel('Rinf')
axis1.set_ylabel('Rinf Value')
axis1.set_title('Rinf Values from DRT Data')
axis1.set_xscale('log')
axis1.set_yscale('log')

axis2.set_xlabel('C0')
axis2.set_ylabel('C0 Value')
axis2.set_title('C0 Values from DRT Data')
axis2.set_xscale('log')
axis2.set_yscale('log')



## Peak Deconvolution

### Draft

In [371]:

from statsmodels.nonparametric.smoothers_lowess import lowess
def feature_Peak(DRTdata_list):
    '''==================================================
        Calculate Rinf from DRTdata_list
        Parameter: 
            DRTdata_list: list of DRT data - day x sample x [T,R,C]
        Return:
            Rinf_list: list of Rinf values
        ==================================================
    '''
    PeakR_list = []
    PeakC_list = []
    for ch_drt in DRTdata_list:
        PeakR = np.concatenate([i[1,1:-1] for i in ch_drt])
        PeakC = np.concatenate([i[2,1:-1] for i in ch_drt])
        
        PeakR_list.append(PeakR)
        PeakC_list.append(PeakC)

    # return np.concatenate(Rinf_list)
    return PeakR_list, PeakC_list



def DRT_Loess(DRTdata):
    '''==================================================
        DRT Analysis with Loess
        Parameter: 
            DRTdata:    list of tuples (tau_i, R_i, C_i) for each sample
        Returen:
            DRTdata_Loess:    Loess smoothed DRT data
        ==================================================
    '''

    _tau_i  = np.concatenate([i[0,1:-1] for i in DRTdata])
    _R_i    = np.concatenate([i[1,1:-1] for i in DRTdata])
    _C_i    = np.concatenate([i[2,1:-1] for i in DRTdata])
    # _tau_i  = np.concatenate([i[0,2:-2] for i in DRTdata])
    # _R_i    = np.concatenate([i[1,2:-2] for i in DRTdata])
    # _C_i    = np.concatenate([i[2,2:-2] for i in DRTdata])

    # _order  = _tau_i.argsort()
    # _tau_i  = _tau_i[_order]
    # _R_i    = _R_i[_order]
    # _C_i    = _C_i[_order]

    x_log = np.log(_tau_i)
    y_log = np.log(_R_i)-np.log(_C_i)
    
    y_log_smooth = lowess(y_log, x_log, frac=0.1, it=3, return_sorted=False)


    R_loess         = np.exp((x_log + y_log_smooth)/2)
    tau_loess       = _tau_i
    C_loess         = _tau_i / R_loess
    DRTdata_Loess   =  np.array([tau_loess, R_loess, C_loess])

    return DRTdata_Loess

In [360]:

PeakR_list, PeakC_list = feature_Peak([DRTdata_list[i] for i in eis_seq])

fig = plt.figure()
axis = fig.add_subplot(111)
axis.scatter(PeakR_list[0]*PeakC_list[0],PeakR_list[0]/PeakC_list[0], s=2)

axis.set_xscale('log')
axis.set_yscale('log')

# x = PeakR_list[0]*PeakC_list[0]
# y = PeakR_list[0]/PeakC_list[0]


In [410]:

DRTdata_Loess = [DRT_Loess(DRTdata_list[i]) for i in eis_seq]


fig = plt.figure()
axis = fig.add_subplot(111)
# axis.scatter(PeakR_list[0]*PeakC_list[0],PeakR_list[0]/PeakC_list[0], s=2, color='gray')
# axis.scatter(DRTdata_Loess[0][1,:]*DRTdata_Loess[0][2,:],DRTdata_Loess[0][1,:]/DRTdata_Loess[0][2,:], s=2)
# axis.scatter(PeakR_list[0]/PeakC_list[0],PeakR_list[0]*PeakC_list[0], s=2, color='gray')
# axis.scatter(DRTdata_Loess[0][1,:]/DRTdata_Loess[0][2,:],DRTdata_Loess[0][1,:]*DRTdata_Loess[0][2,:], s=2)
# axis.scatter(PeakR_list[0]*PeakC_list[0],PeakR_list[0], s=2, color='gray')
# axis.scatter(DRTdata_Loess[0][0,:],DRTdata_Loess[0][1,:], s=2)

axis.scatter(PeakR_list[0]*PeakC_list[0],np.abs(PeakR_list[0] + 1j/PeakC_list[0]), s=2, color='gray')
# axis.scatter(DRTdata_Loess[0][0,:],DRTdata_Loess[0][1,:], s=2)


axis.set_xscale('log')
axis.set_yscale('log')

x = DRTdata_Loess[0][1,:]*DRTdata_Loess[0][2,:]
y = DRTdata_Loess[0][1,:]/DRTdata_Loess[0][2,:]


In [405]:
1+1j

(1+1j)

In [379]:
import numpy as np
from scipy.interpolate import interp1d

# 原始数据 (x, y)，确保是 ndarray 且去除 0

x_log = np.log10(x)
y_log = np.log10(y)
mask = np.isfinite(x_log) & np.isfinite(y_log)
x_log, y_log = x_log[mask], y_log[mask]

# 重采样
x_uniform = np.linspace(x_log.min(), x_log.max(), 1000)
f_interp = interp1d(x_log, y_log, kind='linear', fill_value='extrapolate')
y_uniform = f_interp(x_uniform)


fig = plt.figure()
axis = fig.add_subplot(111)
axis.scatter(x_uniform,y_uniform, s=2)

# axis.set_xscale('log')
# axis.set_yscale('log')


In [388]:
import numpy as np
import matplotlib.pyplot as plt
from lmfit.models import GaussianModel

# 合成示例数据
# x = np.linspace(0, 20, 500)
# np.random.seed(0)
# y = (1.5 * np.exp(-(x - 6)**2 / (2 * 1.2**2)) +
#      2.0 * np.exp(-(x - 13)**2 / (2 * 1.5**2)) +
#      0.05 * np.random.randn(len(x)))


# x = np.log(PeakR_list[0]*PeakC_list[0])
# y = np.log(PeakR_list[0]/PeakC_list[0])
# y = y[x.argsort()]
# x = x[x.argsort()]

x = x_uniform
y = y_uniform
# x = x_uniform
# y = np.power(10,y_uniform)

# 模型定义（两个高斯）
model = GaussianModel(prefix='g1_') + GaussianModel(prefix='g2_')

# 参数初始值估计
# params = model.make_params()
# params['g1_center'].set(-13)
# params['g1_sigma'].set(1)
# params['g1_amplitude'].set(14)

# params['g2_center'].set(13)
# params['g2_sigma'].set(1)
# params['g2_amplitude'].set(2)

# 拟合
result = model.fit(y, params, x=x)

# 拟合结果和每个成分画图
components = result.eval_components(x=x)

plt.plot(x, y, 'b', label='data')
plt.plot(x, result.best_fit, 'r-', label='fit')
plt.plot(x, components['g1_'], 'g--', label='g1')
plt.plot(x, components['g2_'], 'm--', label='g2')
plt.legend()
plt.show()


c:\Users\Baihm\anaconda3\envs\EISNN\Lib\site-packages\uncertainties\core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")


In [392]:
from sklearn.mixture import GaussianMixture

X = x_uniform.reshape(-1, 1)
Y = y_uniform.reshape(-1, 1)

gmm = GaussianMixture(n_components=3).fit(X, Y)
# 拟合值
y_fit = np.exp(gmm.score_samples(X))

plt.plot(x_uniform, y_fit, 'b', label='data')
plt.plot(x_uniform, y_uniform, 'b', label='data')
plt.legend()
plt.show()


# Data Summary

## Definition

In [411]:
def SearchELE(rootPath, ele_pattern = re.compile(r"(.+?)_归档")):
    '''==================================================
        Search all electrode directories in the rootPath
        Parameter: 
            rootPath: current search path
            ele_pattern: electrode dir name patten
        Returen:
            ele_list: list of electrode directories
        ==================================================
    '''
    ele_list = []
    for i in os.listdir(rootPath):
        _path = os.path.join(rootPath, i)
        if os.path.isdir(_path):
            match_ele = ele_pattern.match(i)
            if match_ele:
                ele_list.append([_path, match_ele.group(1)])
            else:
                ele_list.extend(SearchELE(_path, ele_pattern))

    return ele_list



def Load_Single(ele_id, rootPath, DATA_SUFFIX):
    fd_pt = os.path.join(f"{rootPath}/{ele_id}_归档", DATA_SUFFIX, f"{ele_id}_{DATA_SUFFIX}.pt")
    if not os.path.exists(fd_pt):
        logger.warning(f"{fd_pt} does not exist")
        return None
    data_pt = torch.load(fd_pt, weights_only=False)
    _data_group = data_pt["data_group"]

    return _data_group



def DRT_Plot_Batch(fig, DRTdata_list, EISdata_list, Loess_list, eis_seq):
    
    axis = [0] * 6
    axis[0] = fig.add_subplot(2,3,1)    # Nyquist Plot
    axis[1] = fig.add_subplot(2,3,2)    # Bode Plot (Magnitude)
    axis[2] = fig.add_subplot(2,3,3)    # Bode Plot (Phase)
    axis[3] = fig.add_subplot(2,3,4)    # Text
    axis[4] = fig.add_subplot(2,3,5)    # DRT (RC)
    axis[5] = fig.add_subplot(2,3,6)    # DRT (Rτ)


    text_axis = axis[3]
    text_axis.axis('off')

    _s       = 2
    _alpha   = 0.7

    cmap = plt.colormaps.get_cmap('rainbow_r')
    for i in range(len(EISdata_list)):
        if i in eis_seq:
            ch_eis      = EISdata_list[i][0]
            ch_drt      = DRTdata_list[i]
            ch_loess    = Loess_list[i]

            # ch_R    = np.array([i[1:,0] for i in ch_drt])
            # ch_C    = np.array([i[1:,-1] for i in ch_drt])
            ch_R    = np.concatenate([i[1,:] for i in ch_drt])
            ch_C    = np.concatenate([i[2,:] for i in ch_drt])

            _color  = cmap(i/len(EISdata_list))

            axis[0].plot(ch_eis[1,:], -ch_eis[2,:], color = _color, linewidth=2)
            axis[1].loglog(ch_eis[0,:], np.abs(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2)
            axis[2].semilogx(ch_eis[0,:], np.rad2deg(np.angle(ch_eis[1,:]+1j*ch_eis[2,:])), color = _color, linewidth=2)

            axis[4].scatter(ch_R, ch_C, s=_s, alpha=_alpha, color=_color, label=f'ch[{i:03d}]')
            axis[5].scatter(ch_loess[0,:], ch_loess[1,:], s=_s, alpha=_alpha, color=_color, label=f'ch[{i:03d}]')



    axis[0].set_aspect('equal', adjustable='datalim')
    axis[4].set_xscale('log')
    axis[4].set_yscale('log')
    axis[5].set_xscale('log')
    axis[5].set_yscale('log')


    return text_axis




## Load Data

In [412]:

rootPath = "D:/Baihm/EISNN/Archive/"
ele_list = SearchELE(rootPath)
DATASET_SUFFIX = "Outlier_Ver04"

# rootPath = "D:/Baihm/EISNN/Archive_New/"
# ele_list = SearchELE(rootPath)
# DATASET_SUFFIX = "Outlier_Ver04"

# rootPath = "D:/Baihm/EISNN/Invivo/"
# ele_list = SearchELE(rootPath, re.compile(r"(.+?)_Ver02"))
# DATASET_SUFFIX = "Outlier_Ver04"


n_ele = len(ele_list)
logger.info(f"Search in {rootPath} and find {n_ele:03d} electrodes")

2025-07-23 21:11:15.853 | INFO     | __main__:<module>:15 - Search in D:/Baihm/EISNN/Archive/ and find 218 electrodes


In [414]:
ele_id = '06017758'
ch_id = 96     # Perfect of Perfect

# ele_id = '09290511'
# ch_id = 13    # Up & Down, 2 outliers
# ch_id = 21    # Normal + 2 outlier
# ch_id = 41    # Normal + 2 outlier - *(Hard To Tell)
# ch_id = 79    # 3-class, What a mess


# ele_id = '01037160'
# ch_id = 20  # Normal to Short, Same to GPR  
# ch_id = 89  # Same to GPR  
# ch_id = 7  # Normal Example


# ele_id = '05087163'
# ch_id = 7   # one outlier
# ch_id = 50  # No outlier but in two Phases
# ch_id = 55  # One outlier &wired end point
# ch_id = 114 # Open Circuit with on outpler



# ele_id = '02067447'
# ch_id = 68  # Short all the time


# ele_id = '11057712'
# ch_id = 106    # Very Good Electrode with 1 hidden outlier, and one phase shift



# ele_id = '10057084'
# ch_id = 16    # Totaly Mess
# ch_id = 18    # Totaly Mess




Loe_Data = Load_Single(ele_id, rootPath, DATA_SUFFIX = f"{DATASET_SUFFIX}_DRTLoe_Ver02")
Loe_Data = Loe_Data[ch_id]



DRTdata_list    = Loe_Data['DRTlist']  
EISdata_list    = Loe_Data['EISlist']  
Loess_list      = Loe_Data['Loesslist']
eis_seq         = Loe_Data['eis_seq']     
eis_short       = Loe_Data['seq_short']  


# fig = plt.figure(figsize=(16, 9), constrained_layout=True)
# text_axis = DRT_Plot_Batch(fig, DRTdata_list, EISdata_list, Loess_list, eis_seq)

